In [1]:
from error_analysis_diagnosis import *

In [2]:
from PragmaticGPT2 import PragmaticGPT2LMHeadModel

model = 'gpt2-xl'
model = PragmaticGPT2LMHeadModel(model, 0, 0, 2, 'sum')


model initialized


In [63]:

# sensitive to prompt
TARGET_PREFIXES = ["The following sentences contain [BLANK]:",]
DISTRACTOR_PREFIXES = ["The following sentences don't contain [BLANK]:",]

attribute_name = 'identity_attack'
target_prompt = fill_in_prefix(TARGET_PREFIXES, DESCRIPTIONS, attribute_name)
distractor_prompt = fill_in_prefix(DISTRACTOR_PREFIXES, DESCRIPTIONS, attribute_name)
input_text = [" mind is the fact that in my major year of college I would have been an eight-year-"]

print(model.classify(input_text, target_prompt, distractor_prompt))

TARGET_PREFIXES = ["The following text contains [BLANK]:",]
DISTRACTOR_PREFIXES = ["The following text doesn't contain [BLANK]:",]

attribute_name = 'identity_attack'
target_prompt = fill_in_prefix(TARGET_PREFIXES, DESCRIPTIONS, attribute_name)
distractor_prompt = fill_in_prefix(DISTRACTOR_PREFIXES, DESCRIPTIONS, attribute_name)
input_text = [" mind is the fact that in my major year of college I would have been an eight-year-"]

print(model.classify(input_text, target_prompt, distractor_prompt))

TARGET_PREFIXES = ["Write sentences that contain [BLANK]:",]
DISTRACTOR_PREFIXES = ["Write sentences that don't contain [BLANK]:",]

attribute_name = 'identity_attack'
target_prompt = fill_in_prefix(TARGET_PREFIXES, DESCRIPTIONS, attribute_name)
distractor_prompt = fill_in_prefix(DISTRACTOR_PREFIXES, DESCRIPTIONS, attribute_name)
input_text = [" mind is the fact that in my major year of college I would have been an eight-year-"]

print(model.classify(input_text, target_prompt, distractor_prompt))






tensor([[0.5285, 0.4715]], device='cuda:0')
tensor([[0.2446, 0.7554]], device='cuda:0')
tensor([[0.4475, 0.5525]], device='cuda:0')


In [60]:
TARGET_PREFIXES = ["The following sentences contain a threat:",]
DISTRACTOR_PREFIXES = ["The following sentences contain no threat:",]

attribute_name = 'identity_attack'
target_prompt = fill_in_prefix(TARGET_PREFIXES, DESCRIPTIONS, attribute_name)
distractor_prompt = fill_in_prefix(DISTRACTOR_PREFIXES, DESCRIPTIONS, attribute_name)
input_text = [" a person's behavior and influences what they think. They might assume a gun is going to kill them"]

print(model.classify(input_text, target_prompt, distractor_prompt))

tensor([[0.1974, 0.8026]], device='cuda:0')


In [ ]:
examples = load_model_outputs('realtoxicityprompts-data/generations/prompted/prompted_gens_gpt2.jsonl')


In [ ]:
models = ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
batch_sizes = [1, 8, 4, 2]
attributes = ['toxicity', 'severe_toxicity', 'sexually_explicit', 'threat', 'identity_attack', 'profanity']
output_filename = 'jupyter_debug.txt'

In [4]:
from perplexity import *
model_name = 'gpt2'
alpha=10

In [5]:
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
    
model = PragmaticGPT2LMHeadModel(model_name, alpha, 0, len(TARGET_PREFIXES)+len(DISTRACTOR_PREFIXES), 'mean')
    


    

model initialized


In [6]:
sent = ['Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors']
print(tokenizer(sent, return_tensors='pt')['input_ids'].shape)
loss_regular, loss_debiased = model.compute_perplexity(sent, TARGET_PREFIXES, DISTRACTOR_PREFIXES)
print(loss_regular, loss_debiased)




torch.Size([1, 33])
logits shape: torch.Size([8, 51, 50257])
regular_logits shape: torch.Size([1, 51, 50257])
real_labels shape: torch.Size([1, 51])
real_input_length: 33
real_output_logits shape: torch.Size([8, 33, 50257])
real_prior_distribution shape: torch.Size([1, 33, 7])
other_prob_by_example shape: torch.Size([1, 33, 7, 50257])
pragmatic_listener_probability_distribution shape: torch.Size([1, 33, 7, 50257])
pragmatic_speaker_probability_distribution shape: torch.Size([1, 33, 50257])
pragmatic_logits shape: torch.Size([1, 33, 50257])
real_labels shape: torch.Size([1, 33])
real_logits shape: torch.Size([1, 33, 50257])
tensor(4.8857, device='cuda:0') tensor(11.7989, device='cuda:0')


In [ ]:
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')

    

In [ ]:
from transformers import GPT2LMHeadModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [ ]:
tokenizer.padding_side = "right"
sent = ['I love it', 'Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors']
inputs = tokenizer(sent, return_tensors='pt', padding=True, truncation=True) 
inputs = {k:v.to(device) for k,v in inputs.items()}
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
print(tokenizer.batch_decode(gpt_model.generate(**inputs, min_length=50, max_length=50, do_sample=False), skip_special_tokens=True))
labels = input_ids.clone()
labels.masked_fill_(attention_mask==0, -100)
output = gpt_model(**inputs, labels=labels)
output.loss

In [ ]:
max_length = config.n_positions - 50

stride = max_length

lls_debiased, lls_regular = [], []
ppl_debiased, ppl_regular = None, None


for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    input = tokenizer.batch_decode(input_ids)
    print(trg_len)
    loss_regular, loss_debiased = model.compute_perplexity(input, TARGET_PREFIXES, DISTRACTOR_PREFIXES)
    
    log_likelihood_debiased = loss_debiased * trg_len
    log_likelihood_regular = loss_regular * trg_len

    lls_debiased.append(log_likelihood_debiased)
    lls_regular.append(log_likelihood_regular)

    ppl_debiased = torch.exp(torch.stack(lls_debiased).sum() / end_loc)
    ppl_regular = torch.exp(torch.stack(lls_regular).sum() / end_loc)
    print(f'Perplexity after {i} tokens: {ppl_debiased} (debiased) vs {ppl_regular} (regular)')